# Training a MLP using backpropagation

* **part 1** we train using `MLP`. 
* **part 2** we train using `Sequential` and `Linear`. 
* **part 3** we train using `Sequential`, `Linear` and `Conv2D`. 

In [1]:
import jax
from jax import numpy as jnp
from jax import grad, jit
from jax_forward import ReLU, LogSoftmax, MLP, Module, Sequential, Linear, Conv2D, MaxPool2D, Flatten
from time import time
dspath = "../datasets"
jax.config.update('jax_platform_name', 'cpu')

## Part 1
We first define useful parameters that we are going to use later. Note that our model is a MLP imported from `net.py`

In [2]:
# hyperparameters
layer_sizes = [784, 512, 256, 10]
step_size = 0.01
num_epochs = 5
batch_size = 128
n_targets = 10
key = jax.random.PRNGKey(0)
function = ReLU()
log_softmax = LogSoftmax()

# to create a model is as simple as that
model = MLP(layer_sizes, function, key)
params = model.params # we need to extract parameters

Helper functions, trainloader and other stuff...

In [3]:
# some helper functions
def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(model(images,params), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = model(images, params)
  preds = log_softmax(preds)
  return -jnp.mean(preds * targets)

#@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]


import numpy as np
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class NumpyLoader(data.DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))


class Cast(object):
  def __call__(self, pic):
    return np.array(pic, dtype=jnp.float32)

# Define our dataset, using torch datasets
mnist_dataset = MNIST(dspath, download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)

# Get the full train dataset (for checking accuracy while training)
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST(dspath, download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

/home/francesco/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/francesco/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/francesco/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/francesco/.local/lib/python3.10/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Here we train using backpropagation

In [4]:
import time

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 24.71 sec
Training set accuracy 0.9044666886329651
Test set accuracy 0.911899983882904
Epoch 1 in 14.61 sec
Training set accuracy 0.9273666739463806
Test set accuracy 0.9305999875068665
Epoch 2 in 13.67 sec
Training set accuracy 0.9414666891098022
Test set accuracy 0.9399999976158142
Epoch 3 in 11.93 sec
Training set accuracy 0.9496999979019165
Test set accuracy 0.9475999474525452
Epoch 4 in 13.27 sec
Training set accuracy 0.9562833309173584
Test set accuracy 0.9531999826431274


## Part 2
We first define useful parameters that we are going to use later. Note that our model is a costum model built using `Sequential` and `Linear`.

In [5]:
class myModel(Module):
    def __init__(self):
        self.layers = Sequential([
            Linear(28*28,512,key),
            function,
            Linear(512,256,key),
            function,
            Linear(256,10,key)
        ])
        self.params = self.layers.generate_parameters()

    def forward(self, data, params):
        return self.layers(data, params)


# we define the model as simple as that
model = myModel()
params = model.params

Let's train

In [6]:
for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 14.20 sec
Training set accuracy 0.909416675567627
Test set accuracy 0.9139999747276306
Epoch 1 in 12.16 sec
Training set accuracy 0.9317833185195923
Test set accuracy 0.9317999482154846
Epoch 2 in 12.25 sec
Training set accuracy 0.9434333443641663
Test set accuracy 0.9411999583244324
Epoch 3 in 12.09 sec
Training set accuracy 0.951449990272522
Test set accuracy 0.9483999609947205
Epoch 4 in 12.97 sec
Training set accuracy 0.9581000208854675
Test set accuracy 0.9537999629974365


## Part 3
We first define useful parameters that we are going to use later. Note that our model is a costum model built using `Sequential`, `Linear` and `Conv2D` base on this [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [7]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torchvision
from torch import transpose

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), Cast(),])

cifar_dataset = CIFAR10(dspath, download=True,train=True,
                        transform=transform)
training_generator = NumpyLoader(
    cifar_dataset, batch_size=batch_size, num_workers=0)

# Get the full train dataset (for checking accuracy while training)
train_images = jnp.array([item[0] for item in cifar_dataset])
train_labels = one_hot(np.array(cifar_dataset.targets), n_targets)

# Get full test dataset
cifar_dataset_test = CIFAR10(dspath, download=True, train=False,
                             transform=transform)
test_images = jnp.array([item[0] for item in cifar_dataset_test])
test_labels = one_hot(np.array(cifar_dataset.targets), n_targets)


Files already downloaded and verified


Let's define the model

In [ ]:
class myModel(Module):
    def __init__(self):
        self.layers = Sequential([
            Conv2D(3, 6, 5, 1, 'VALID', key),
            function,
            MaxPool2D(2, 2),
            Conv2D(6, 16, 5, 1, 'VALID', key),
            function,
            MaxPool2D(2, 2),
            Flatten(),
            Linear(16 * 5 * 5, 120, key),
            function,
            Linear(120, 84, key),
            function,
            Linear(84, 10, key)
        ])
        self.params = self.layers.generate_parameters()

    def forward(self, data, params):
        return self.layers(data, params)


# we define the model as simple as that
model = myModel()
params = model.params


In [ ]:
from time import time
step_size = 0.001
for epoch in range(num_epochs):
  start_time = time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time() - start_time

  train_acc = accuracy(params, train_images[:10000], train_labels[:10000])
  test_acc = accuracy(params, test_images[:10000], test_labels[:10000])
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 48.65 sec
Training set accuracy 0.1264999955892563
Test set accuracy 0.09779999405145645
Epoch 1 in 45.66 sec
Training set accuracy 0.1582999974489212
Test set accuracy 0.09629999846220016
